In [ ]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate
import miou_eval as mi

In [ ]:
class alchemy(nn.Module):
    def __init__(self,**kwargs) -> None:
        super(alchemy, self).__init__()
        self.vis = False
        self.maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,trainable_backbone_layers=5,**kwargs)
        in_features = self.maskrcnn.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
        self.maskrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one

    def forward(self, images, targets=None):
        ret = self.maskrcnn(images,targets)
        return ret

In [14]:
device = 'cpu'

In [15]:
model1 = alchemy(box_nms_thresh=0.5)
model1.to('cpu')
model1.load_state_dict(torch.load("alchemy_276_0.915.torch",map_location='cpu'))
model1.eval()
#model1 = model
for im_name in os.listdir("./submission/x/"):
    imgtmp = cv2.imread("./submission/x/"+im_name,1)
    imgtmp = cv2.cvtColor(imgtmp, cv2.COLOR_BGR2LAB ).transpose(2,0,1)
    imgtmp = torch.as_tensor(imgtmp, dtype=torch.float32)
    imgtmp = imgtmp/255
    imgtmp = imgtmp.to(device)
    output = model1([imgtmp])
    prefix = im_name[0:-4]
    ref = 0
    for i in range(output[0]['masks'].shape[0]):
        print(output[0]['scores'][i].item())
        it = torch.clone(output[0]['masks'][i])
        it = it.detach().to('cpu').numpy()
        it = it.squeeze(0)
        it = (it>=0.5).astype(float)
        it = it*255
        cv2.imwrite("./submission/y/"+prefix+"_"+str(i)+".bmp",it)

0.9995263814926147
0.9994202852249146
0.9992579817771912
0.9991790652275085
0.9990719556808472
0.9990567564964294
0.9989919066429138
0.998970627784729
0.9989171028137207
0.998889148235321
0.9988886713981628
0.9988693594932556
0.9988528490066528
0.9988011121749878
0.9987826943397522
0.9983237385749817
0.9982313513755798
0.9981371164321899
0.9978227615356445
0.9974408149719238
0.9973278045654297
0.9966394901275635
0.9963879585266113
0.9950350522994995
0.9942626357078552
0.9924045205116272
0.9899167418479919
0.9897165894508362
0.9799193143844604
0.9770568013191223
0.9172906875610352
0.3405322730541229
0.9997594952583313
0.9995993971824646
0.9995929598808289
0.9995700716972351
0.9995219707489014
0.9995136260986328
0.9994340538978577
0.9992690682411194
0.9992396831512451
0.9992068409919739
0.9991626739501953
0.9989292025566101
0.9989198446273804
0.9988510608673096
0.9979945421218872
0.9978118538856506
0.9977214932441711
0.9975503087043762
0.9967825412750244
0.995180606842041
0.9950820207595

In [16]:
! make sub

cp ./submission/x/* ./submission/y
python ../reference/submission.py -s ./submission/y -d ./ -nu 255
  0%|                                                   | 0/554 [00:00<?, ?it/s]../reference/submission.py:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = cv2.resize(mask.astype(np.uint8),(sz[1],sz[0]),interpolation=cv2.INTER_NEAREST).astype(np.bool)
100%|█████████████████████████████████████████| 554/554 [03:45<00:00,  2.46it/s]
554
saved: ./submission.txt
zip subby.zip submission.txt 
  adding: submission.txt (deflated 71%)
